In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import copy
import json

In [2]:
df = pd.read_excel("./18_data_laundering.xlsx")
word_dict_10 = {}
with open("../word_dict_10.json", "r") as outfile:
    #print(outfile.readfile())
    word_dict_10 = json.load(outfile)
word_dict_5 = {}
with open("../word_dict_5.json", "r") as outfile:
    #print(outfile.readfile())
    word_dict_5 = json.load(outfile)

In [3]:
#df.rename(columns={"id": "id", "text": "text", "view": "view", "vote": "vote", "chapter": "chapter", 
#                   1.0: "1", 2.0: "2", 3.0: '3', 4.0: "4", 5.0: "5", 6.0: "6", 7.0: "7", 8.0: "8"})

In [4]:
df.columns

Index(['id', 'text', 'view', 'vote', 'chapter', '1', '2', '3', '4', '5', '6',
       '7', '8'],
      dtype='object')

# Clean stopword: clean words that appears a lot in all true tags

In [5]:
top_20_total = []

for tag in range(1, 9):
    true_df = df.loc[df[str(tag)] == True].reset_index()
    false_df = df.loc[df[str(tag)] == False].reset_index()

    true_agg_words = []
    false_agg_words = []

    for i in range(len(true_df)):
        text = true_df.at[i, "text"]
        true_agg_words.extend(text.split(' '))

    for i in range(len(false_df)):
        text = false_df.at[i, "text"]
        false_agg_words.extend(text.split(' '))

    true_counter = Counter(true_agg_words)
    false_counter = Counter(false_agg_words)

    #all_counter = copy.deepcopy(true_counter)
    #for key, item in all_counter.items():
    #    if key in false_counter.keys():
    #        all_counter[key] -= false_counter[key]
    #        del true_counter[key]
    #        del false_counter[key]
            
    top_20_true = [key for key, _ in true_counter.most_common(20)]
    #print(f"Tag: {tag}")
    #print(top_20_true)
    top_20_total.extend(top_20_true)

In [6]:
Counter(top_20_total)

Counter({'1': 8,
         'truyện': 8,
         's': 7,
         't': 7,
         'edit': 7,
         'tác_giả': 7,
         'ta': 7,
         'love': 6,
         'life': 5,
         'stori': 5,
         'đi': 5,
         'year': 5,
         'nữ': 5,
         'chương': 5,
         'find': 4,
         'book': 4,
         'hắn': 4,
         'nàng': 3,
         'want': 3,
         'yêu': 3,
         '2': 3,
         'know': 3,
         'girl': 3,
         'hoàn': 3,
         'nam': 3,
         'xuyên': 3,
         'world': 2,
         'like': 2,
         'x': 2,
         'famili': 1,
         'read': 1,
         'start': 1,
         'thụ': 1,
         'công': 1,
         'mỹ': 1,
         'đam': 1,
         'chủ': 1,
         'boy': 1,
         'thing': 1,
         'tình': 1,
         'giới': 1,
         'school': 1,
         'hiện_đại': 1,
         'sinh': 1,
         'hệ_thống': 1,
         'gia': 1,
         'cổ_đại': 1,
         'sủng': 1,
         'trọng': 1,
         'chết': 1,
    

In [7]:
stop_words_final = ["stori", "đi", "year", "chương", "find", "book"]
stop_words_final.extend([key for key, item in Counter(top_20_total).items() if item >= 7])

In [8]:
def clean_stopword(description):
    old_description = description.split(" ")
    new_description = []
    
    for item in old_description:
        if item in stop_words_final:
            continue
            
        new_description.append(item)
        
    return " ".join(new_description)

In [9]:
df["text"] = df["text"].apply(clean_stopword)

# Clean unnecessary words

In [10]:
df_10 = copy.deepcopy(df)
    
def clean_df_10(text):
    description = text.split(" ")
    
    new_description = []
    
    for item in description:
        if item in word_dict_10.keys():
            new_description.append(item)
            
    return " ".join(new_description)
    
df_10["text"] = df_10["text"].apply(clean_df_10)
df_10

,id,text,view,vote,chapter,1,2,3,4,5,6,7,8
0,0.0,vampir sói 12 chòm sống tình_cảm 12 chòm câu k...,69463.0,3355.0,32.0,False,True,True,False,False,False,False,False
1,1.0,nữ tác hoạ cốt kỷ 21 nàng nổi_tiếng giới xuyên...,3529078.0,128419.0,160.0,False,True,False,False,False,False,False,False
2,2.0,share sở,87261.0,2780.0,13.0,False,True,False,False,False,False,False,False
3,3.0,tình_yêu giết góc gái hoàn_thành câu chữ chút ...,34795.0,2049.0,17.0,True,True,False,False,False,False,False,False
4,4.0,hán_việt tuyệt_vọng giáo thất vong kỳ ly vốn t...,25746.0,1068.0,NaN,False,True,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2885,2970.0,phong đỏ chia lặng gió phong written,1090.0,150.0,NaN,False,False,False,True,False,False,False,False
2886,2971.0,lee x home mệt_mỏi nhẹ_nhàng ôm bao mệt_mỏi ta...,452.0,64.0,5.0,True,False,False,True,False,False,False,False
2887,2972.0,tình_cờ yêu ooc nguyên_tác coi đường đường vị ...,81.0,4.0,1.0,True,False,False,False,False,False,False,False
2888,2973.0,cute x reader author wattpad fandom thương cou...,NaN,NaN,NaN,False,False,True,False,False,False,False,False


In [11]:
df_5 = copy.deepcopy(df)
    
def clean_df_5(text):
    description = text.split(" ")
    
    new_description = []
    
    for item in description:
        if item in word_dict_5.keys():
            new_description.append(item)
            
    return " ".join(new_description)
    
df_5["text"] = df_5["text"].apply(clean_df_5)
df_5

,id,text,view,vote,chapter,1,2,3,4,5,6,7,8
0,0.0,vampir sói 12 chòm sống tình_cảm 12 chòm câu k...,69463.0,3355.0,32.0,False,True,True,False,False,False,False,False
1,1.0,nữ tác hoạ cốt kỷ 21 nàng nổi_tiếng giới xuyên...,3529078.0,128419.0,160.0,False,True,False,False,False,False,False,False
2,2.0,share sở,87261.0,2780.0,13.0,False,True,False,False,False,False,False,False
3,3.0,nạn_nhân tình_yêu giết góc gái hoàn_thành câu ...,34795.0,2049.0,17.0,True,True,False,False,False,False,False,False
4,4.0,lớp_học hán_việt tuyệt_vọng giáo thất vong kỳ ...,25746.0,1068.0,NaN,False,True,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2885,2970.0,phong đỏ chia lặng gió lá phong written woo,1090.0,150.0,NaN,False,False,False,True,False,False,False,False
2886,2971.0,lee minho x hyunjin home mệt_mỏi nhẹ_nhàng ôm ...,452.0,64.0,5.0,True,False,False,True,False,False,False,False
2887,2972.0,tình_cờ yêu ooc lệch nguyên_tác coi đường đườn...,81.0,4.0,1.0,True,False,False,False,False,False,False,False
2888,2973.0,cute x reader author wattpad fandom thương cou...,NaN,NaN,NaN,False,False,True,False,False,False,False,False


In [12]:
df_10.to_csv("../data_10_freq.csv", index=False)
df_5.to_csv("../data_5_freq.csv", index=False)

In [13]:
df["text"].isna().any()

False

In [14]:
df.to_csv("../data_no_freq.csv", index=False)

In [15]:
for i in range(1, 9):
    classes = str(i)
    true_label = df[classes].sum()
    num_samples = len(df)
    print("Class {} true label percentage: {:.2f}".format(i, float(true_label) / float(num_samples) * 100))

Class 1 true label percentage: 51.00
Class 2 true label percentage: 33.81
Class 3 true label percentage: 34.36
Class 4 true label percentage: 26.30
Class 5 true label percentage: 10.59
Class 6 true label percentage: 20.28
Class 7 true label percentage: 5.19
Class 8 true label percentage: 5.95
